### Run Eval

In [3]:
import os
import os.path as osp
if osp.basename(os.getcwd()) != 'VideoPose3D': os.chdir('../VideoPose3D')
import torch
import pickle
from common.loss import mpjpe

In [20]:
with open('../notebooks/benchmark.pkl', 'rb') as f:
    test_generator, model_pos = pickle.load(f)

In [3]:
model_pos.eval()
epoch_loss_3d_valid = 0
N = 0

for cam, batch, batch_2d in test_generator.next_epoch():
    inputs_3d = torch.from_numpy(batch.astype('float32'))
    inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
    if torch.cuda.is_available():
        inputs_3d = inputs_3d.cuda()
        inputs_2d = inputs_2d.cuda()
        model_pose = model_pos.cuda()
    inputs_traj = inputs_3d[:, :, :1].clone()
    inputs_3d[:, :, 0] = 0
    predicted_3d_pos = model_pos(inputs_2d)
    loss_3d_pos = mpjpe(predicted_3d_pos, inputs_3d)
    epoch_loss_3d_valid += inputs_3d.shape[0] * inputs_3d.shape[1] * loss_3d_pos.item()
    N += inputs_3d.shape[0] * inputs_3d.shape[1]
    
print('Total eval loss is:', 1000 * epoch_loss_3d_valid / N)

Total eval loss is: 55.05829741033194


### Generate Pickle File

In [4]:
import sys
import torch
import os
import os.path as osp
import pickle
if osp.basename(os.getcwd()) != 'VideoPose3D': os.chdir('../VideoPose3D')
from run import test_generator, model_pos
# use the below command line argument for test run

Command Line Arguments: -k cpn_ft_h36m_dbb -arc 3,3,3 -ch 256 -c checkpoint --evaluate arc_333_ch_256_epoch_80.bin
Namespace(actions='*', architecture='3,3,3', batch_size=1024, benchmark=False, bone_length_term=True, by_subject=False, causal=False, channels=256, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='h36m', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='arc_333_ch_256_epoch_80.bin', export_training_curves=False, keypoints='cpn_ft_h36m_dbb', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=False, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action=None, viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export=None, viz_limit=-1, viz_no_ground_truth=False, viz_output=None, viz_size=5, viz_skip=0, viz_subject=None, viz_video=None, warmup=1)
Loading data

-k cpn_ft_h36m_dbb -arc 3,3,3 -ch 256 -c checkpoint --evaluate arc_333_ch_256_epoch_80.bin

In [5]:
with open('../notebooks/benchmark.pkl', 'wb') as f:
    pickle.dump((test_generator, model_pos), f)